In [ ]:
pip install langchain langchain-google-genai langchain-community requests


In [ ]:
import os

os.environ["GOOGLE_API_KEY"] = "YOUR_GEMINI_API_KEY"
os.environ["WEATHER_API_KEY"] = "YOUR_WEATHERAPI_KEY"


In [ ]:
import os
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate


# ---------------- LLM SETUP ----------------
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0
)

# ---------------- WEATHER TOOL ----------------
@tool
def get_weather(city: str) -> str:
    """
    Fetch current weather for a given city using WeatherAPI.
    """
    api_key = os.environ["WEATHER_API_KEY"]
    url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}"

    response = requests.get(url)
    data = response.json()

    if "error" in data:
        return "Unable to fetch weather information."

    weather = data["current"]
    return (
        f"Temperature: {weather['temp_c']}°C, "
        f"Condition: {weather['condition']['text']}, "
        f"Humidity: {weather['humidity']}%"
    )


# ---------------- ATTRACTION SEARCH TOOL ----------------
search_tool = DuckDuckGoSearchRun()

@tool
def get_top_attractions(city: str) -> str:
    """
    Search for top tourist attractions in a city.
    """
    query = f"Top tourist attractions in {city}"
    return search_tool.run(query)


# ---------------- TOOLS LIST ----------------
tools = [get_weather, get_top_attractions]

# ---------------- PROMPT ----------------
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an intelligent travel assistant. "
     "Use tools to get weather and top tourist attractions."),
    ("human", "{input}")
])

# ---------------- AGENT CREATION ----------------
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

# ---------------- USER INTERACTION ----------------
destination = input("Enter travel destination: ")

response = agent_executor.invoke({
    "input": f"Give me the current weather and top attractions in {destination}"
})

print("\n--- Travel Assistant Output ---\n")
print(response["output"])
